In [2]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import pandas as pd
import uuid   # Unique identifier
import os
import time

C:\Users\Digvijay Singh Dhank\anaconda3\envs\drowsy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 500 --data dataset.yml --weights yolov5s.pt --workers 2

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\Digvijay Singh Dhank/.cache\torch\hub\master.zip
YOLOv5  2022-5-16 Python-3.8.13 torch-1.8.2+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [10]:
cap = cv2.VideoCapture(0)
var = ""
employee_score = 100;
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    cv2.imshow('YOLO', np.squeeze(results.render()))
    df = results.pandas().xyxy[0]
#     if df.name.iloc[0]to_string() == "drowsy":
#         print("wake up")
#     else:
#         print("Job being done")
    d_count=df['name'].str.contains('drowsy').sum()
    if d_count>0:
        employee_score = employee_score - d_count
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
type(var)

In [11]:
employee_score

56